>**기출문제 제9회**
>>**01 작업형 제1유형**

## 옵션 설정

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # 모든 칼럼이 출력되게 조절

## 문제 1. 
- 각 연도 및 성별의 총 대출액의 절댓값 차이를 구한다. 
- 절댓값 차이가 가장 큰 지역코드를 구하시오. 


In [3]:
data1 = pd.read_csv('9_1_1.csv') 
data1.head(5)

,year,gender,지역코드,금액1,금액2
0,2018,0,143,70713,43210
1,2019,0,183,40306,39663
2,2017,0,129,26646,41631
3,2019,0,161,56843,12455
4,2019,0,174,26371,9014


In [33]:
# 금액1과 금액2을 합하여 총 대출액 구하기 
data1['총대출액'] = data1['금액1'] + data1['금액2']

# 연도, 성별, 지역코드별로 총대출액 합계 구하기
grouped = data1.groupby(['year', 'gender', '지역코드'])['총대출액'].sum().reset_index()

# 각 연도 및 성별에 따른 차이 계산 위해 피벗 테이블 생성
data1_pivot = pd.pivot_table(grouped, 
                            values='총대출액',
                            index=['year', '지역코드'],
                            columns='gender',
                            fill_value=0)

# 성별에 따른 절댓값 차이 계산
data1_pivot['차이'] = abs(data1_pivot[0] - data1_pivot[1])

# 차이가 가장 큰 행 찾기
max_diff_row = data1_pivot['차이'].idxmax()

# 해당 행의 데이터 출력
print(data1_pivot.loc[max_diff_row, :])

# 지역코드
answer = max_diff_row[1]  # (year, 지역코드) 튜플에서 지역코드 추출
print(answer)


gender
0     168869.0
1          0.0
차이    168869.0
Name: (2018, 143), dtype: float64
143


## 문제 2. 
- 각 연도별 최대 검거율을 가진 범죄유형을 찾아서 해당 연도 및 유형의 검거건수들의 총합을 구하시오. 
- 검거율 = 검거건수 / 발생건수 

In [36]:
data2 = pd.read_csv('9_1_2.csv')
data2.head(1)

,연도,구분,범죄유형1,범죄유형2,범죄유형3,범죄유형4,범죄유형5,범죄유형6,범죄유형7,범죄유형8,범죄유형9,범죄유형10
0,2000,발생건수,302,635,470,306,271,900,220,814,321,666


In [72]:
# 검거건수와 발생건수 데이터 추출
검거건수_df = data2[data2['구분'] == '검거건수'].set_index('연도')
발생건수_df = data2[data2['구분'] == '발생건수'].set_index('연도')

# 구분 열 제거
검거건수_df = 검거건수_df.drop('구분', axis=1)
발생건수_df = 발생건수_df.drop('구분', axis=1)

# 검거율 계산
검거율 = 검거건수_df / 발생건수_df

# 각 연도별 최대 검거율을 가진 범죄유형 찾기
최대검거율_범죄유형 = 검거율.idxmax(axis=1).reset_index().rename(columns={0: '범죄유형'})

# 연도와 범죄유형에 해당하는 검거건수 찾기
검거건수_데이터 = 검거건수_df.reset_index().melt(
    id_vars='연도', 
    var_name='범죄유형', 
    value_name='검거건수'
)

# 최대 검거율 범죄유형과 검거건수 데이터 병합
결과 = pd.merge(최대검거율_범죄유형, 검거건수_데이터, on=['연도', '범죄유형'])

# 검거건수 합계 계산
answer = 결과['검거건수'].sum()
print('총 검거건수 합계 =', answer)


총 검거건수 합계 = 9879


In [73]:
# 데이터 준비
발생건수_df = data2[data2['구분'] == '발생건수'].set_index('연도')
검거건수_df = data2[data2['구분'] == '검거건수'].set_index('연도')

# 필요없는 '구분' 열 제거
발생건수_df = 발생건수_df.drop('구분', axis=1)
검거건수_df = 검거건수_df.drop('구분', axis=1)

# 데이터를 long 형식으로 변환
발생건수_long = 발생건수_df.reset_index().melt(id_vars='연도', var_name='범죄유형', value_name='발생건수')
검거건수_long = 검거건수_df.reset_index().melt(id_vars='연도', var_name='범죄유형', value_name='검거건수')

# 두 데이터프레임 병합
통합_df = pd.merge(발생건수_long, 검거건수_long, on=['연도', '범죄유형'])

# 검거율 계산
통합_df['검거율'] = 통합_df['검거건수'] / 통합_df['발생건수']

# 연도별 그룹화하여 최대 검거율 찾기
최대검거율_df = 통합_df.loc[통합_df.groupby('연도')['검거율'].idxmax()]

# 검거건수 합계 계산
answer = 최대검거율_df['검거건수'].sum()
print('총 검거건수 합계 =', answer)


총 검거건수 합계 = 9879


## 문제 3. 
- 결측치 처리
    + 평균 만족도 : 결측치는 평균만족도 컬럼의 전체 평균으로 채우시오. 
    + 근속연수 : 결측치는 각 부서와 등급별 평균 근속연수로 채우시도. (평균값의 소수점은 버림 처리)
- 조건에 따른 평균 계산
    + A : 부서가 'HR'이고 등급이 'A'인 사람들의 평균 근속연수를 계산하시오. 
    + B : 부서가 'Sales'이고 등급이 'B'인 사람들의 평균 교육참가횟수를 계산하시오. 
    + A와 B를 더한 값을 구하시오. 

In [77]:
# 데이터 로드
data3 = pd.read_csv('9_1_3.csv')
data3.head(1)

,부서,등급,평균만족도,근속연수,교육참가횟수
0,Finance,C,4.023911,17.0,8.0


In [79]:
data3.isna().sum()

부서         0
등급         0
평균만족도     10
근속연수      10
교육참가횟수     0
dtype: int64

In [81]:
# 결측치 처리
# 1. 평균 만족도 결측치를 전체 평균으로 채우기
평균_만족도 = data3['평균만족도'].mean()
data3['평균만족도'] = data3['평균만족도'].fillna(평균_만족도)

# 2. 근속연수 결측치를 각 부서와 등급별 평균으로 채우기
# 부서와 등급별 평균 근속연수 계산 (소수점 버림)
부서_등급별_평균 = data3.groupby(['부서', '등급'])['근속연수'].transform(lambda x: np.floor(x.mean()))

# 결측치만 채우기 (transform 사용하여 반복문 없이 처리)
mask = data3['근속연수'].isna()
data3.loc[mask, '근속연수'] = 부서_등급별_평균[mask]

data3.isna().sum()

부서        0
등급        0
평균만족도     0
근속연수      0
교육참가횟수    0
dtype: int64

In [82]:
# A: 부서가 'HR'이고 등급이 'A'인 사람들의 평균 근속연수 계산
A = data3[(data3['부서'] == 'HR') & (data3['등급'] == 'A')]['근속연수'].mean()

# B: 부서가 'Sales'이고 등급이 'B'인 사람들의 평균 교육참가횟수 계산
B = data3[(data3['부서'] == 'Sales') & (data3['등급'] == 'B')]['교육참가횟수'].mean()

# A와 B를 더한 값 계산
answer = A + B
print('HR 부서 A등급 평균 근속연수 =', A)
print('Sales 부서 B등급 평균 교육참가횟수 =', B)
print('A + B =', answer)

HR 부서 A등급 평균 근속연수 = 17.625
Sales 부서 B등급 평균 교육참가횟수 = 7.6
A + B = 25.225
